In [15]:
import random
import rsa
import math

def profile(func):
    from functools import wraps

    @wraps(func)
    def wrapper(*args, **kwargs):
        from line_profiler import LineProfiler
        prof = LineProfiler()
        try:
            return prof(func)(*args, **kwargs)
        finally:
            prof.print_stats()

    return wrapper

In [16]:
@profile
def trial_division(n):
    if n % 2 == 0:
        return 2
    else:
        for i in range(3, n, 2):
            if n % i == 0:
                return i


# @profile
def lucky(n):
    d = 1
    while d == 1:
        x = random.randint(1, n - 1)
        d = math.gcd(x, n)
    if d != n:
        return d


# @profile
def rho_random(n):
    d = 1
    while d == 1:
        x = random.randint(1, n - 1)
        y = random.randint(1, n - 1)
        d = math.gcd((x - y) % n, n)
    if d != n:
        return d


# @profile
def pollard_rho_classic(n, seed=2, f=lambda x: x**2 + 1):
    x, d = seed, 1
    prev = [x]
    while True:
        x = f(x) % n
        for v in prev:
            d = math.gcd((x - v) % n, n)
            if d == n:
                return None
            if d != 1:
                return d
        prev.append(x)


@profile
def pollard_rho_floyd(n, fac=0, seed=2, f=lambda x: x**2 + 1):
    x, y, d = seed, seed, 1
    while d == 1:
        x = f(x) % n
        y = f(f(y)) % n
        d = math.gcd((x - y) % n, n)
    if d != n:
        return d


@profile
def pollard_rho_brent(n, seed=2, f=lambda x: x**2 + 1):
    # https://mathworld.wolfram.com/BrentsFactorizationMethod.html -- the most layman explanation i found
    m = 1000
    y = seed
    r, q, d = 1, 1, 1
    while d == 1:
        x = y
        for i in range(r):
            y = f(y) % n

        k = 0
        while k < r and d == 1:
            ys = y
            for i in range(min(m, r - k)):
                y = f(y) % n
                q *= (x - y) % n
            d = math.gcd(q, n)
            k += m
        r *= 2
    if d == n:
        while True:
            ys = f(ys) % n
            d = math.gcd(x - ys, n)
            if d > 1:
                break
    if d != n:
        return d


@profile
def fermat(n):
    a = math.ceil(math.sqrt(n))
    b = a**2 - n
    while not math.sqrt(b).is_integer():
        a += 1
        b = a**2 - n
    return a - math.isqrt(b)


@profile
def fermat_classroom(n):
    a = math.isqrt(n)
    b = 0
    while (ret := a**2 - b**2 - n) != 0:
        if ret > 0:
            b += 1
        else:
            a += 1
    return int(a - b)


In [17]:
q = rsa.prime.getprime(16)
p = rsa.prime.getprime(16)

n = q * p
print("prime", n)
# print(trial_division(n))
# print(lucky(n))
# print(rho_random(n))
# print(pollard_rho_classic(n))
print(pollard_rho_floyd(n))
# print(pollard_rho_brent(n))
# print(fermat(n))
# print(fermat_classroom(n))

# while True:
    # q = rsa.prime.getprime(8)
    # p = rsa.prime.getprime(8)
    # n = q * p
    # f = pollard_rho_classic(n)
    # print(f)
    # if not f:
        # print("prime", n)
        # break

prime 2359617427
Timer unit: 1e-09 s

Total time: 0.00106064 s
File: /tmp/ipykernel_37681/854439106.py
Function: pollard_rho_floyd at line 47

Line #      Hits         Time  Per Hit   % Time  Line Contents
    47                                           @profile
    48                                           def pollard_rho_floyd(n, fac=0, seed=2, f=lambda x: x**2 + 1):
    49         1       2759.0   2759.0      0.3      x, y, d = seed, seed, 1
    50       109      83383.0    765.0      7.9      while d == 1:
    51       109     285113.0   2615.7     26.9          x = f(x) % n
    52       109     438847.0   4026.1     41.4          y = f(f(y)) % n
    53       109     248975.0   2284.2     23.5          d = math.gcd((x - y) % n, n)
    54         1        791.0    791.0      0.1      if d != n:
    55         1        772.0    772.0      0.1          return d

60961


In [18]:
# collision d < n
#######################
# print(pollard_rho_classic(n,f = lambda x: x**2 + 2))
# prime = 143

# 2 6
# 6 38
# 38 16
# 16 115
# 115 71
# 71 38
# 38 16
# 16 115
# 115 71
# 71 38
# 38 16

# 2 6
# 6 5
# 5 5
# 5 5
# 5 5
# 5 5
# 5 5
# 5 5
# 5 5
# 5 5
# 5 5
# 5 5
#######################

# print(pollard_rho_floyd(n))
# 5 26
# 26 15
# 105 26
# 15 15
# 83 26
# 26 15
# 105 26
# 15 15
# 83 26
# 26 15
# 105 26
# 15 15

# 5 4
# 4 4
# 6 4
# 4 4
# 6 4
# 4 4
# 6 4
# 4 4
# 6 4
# 4 4
# 6 4
# 4 4
# 6 4

# collision n == d
#######################
# print(pollard_rho_floyd(n))
# 5 26
# 26 2
# 1 26
# 2 2
# 5 26
# 26 2
# 1 26
# 2 2
# 5 26
# 26 2
# 1 26
# 2 2

# 5 0
# 0 2
# 1 0
# 2 2
# 5 0
# 0 2
# 1 0
# 2 2
# 5 0
# 0 2
# 1 0
# 2 2

# prime 3127
# classic

# 2 5
# 5 26
# 26 677
# 677 1788
# 1788 1151
# 1151 2081
# 2081 2794
# 2794 1445
# 1445 2317
# 2317 2558
# 2558 1681
# 1681 2081
# 2081 2794
# 2794 1445
# 1445 2317
# 2317 2558
# 2558 1681
# 1681 2081
# 2081 2794

# floyd
# 5 26
# 26 1788
# 677 2081
# 1788 1445
# 1151 2558
# 2081 2081
# 2794 1445
# 1445 2558
# 2317 2081
# 2558 1445
# 1681 2558
# 2081 2081
# 2794 1445
# 1445 2558
# 2317 2081
# 2558 1445
# 1681 2558
# 2081 2081

# 5 26
# 26 39
# 41 14
# 39 14
# 38 14
# 14 14
# 38 14
# 14 14
# 38 14
# 14 14
# 38 14
# 14 14
# 38 14
# 14 14
# 38 14
# 14 14

